In [26]:
import pandas as pd
import os
import numpy as np

def load_data(disease, data_dir):
    raw_data = dict()
    raw_data['abundance'] = pd.read_csv(
        os.path.join(data_dir, 'data/abundance/abundance_{}.txt'.format(disease)),
        sep="\t",
        skiprows=1,
        low_memory=False
    )
    raw_data['marker'] = pd.read_csv(
        os.path.join(data_dir, 'data/marker/marker_{}.txt'.format(disease)),
        sep="\t",
        skiprows=1,
        low_memory=False
    )
    return raw_data

In [5]:
for disease in ['Cirrhosis', 'Colorectal', 'IBD', 'Obesity', 'T2D', 'WT2D']:
    raw_data = load_data(disease, '.')
    marker = raw_data['marker'].iloc[209:, 1:].astype('float32')
    marker["sum_row"] = marker.sum(axis=1)
    abundance = raw_data['abundance'].iloc[209:, 1:].astype('float32')
    abundance["sum_row"] = abundance.sum(axis=1)
    print('Min sum row: Abundance {} | Marker {}'.format(abundance["sum_row"].min(), marker["sum_row"].min()))

Min sum row: Abundance 0.0003699999942909926 | Marker 1.0
Min sum row: Abundance 7.000000186963007e-05 | Marker 1.0
Min sum row: Abundance 0.00015999999595806003 | Marker 1.0
Min sum row: Abundance 9.999999747378752e-05 | Marker 1.0
Min sum row: Abundance 0.0002300000051036477 | Marker 1.0
Min sum row: Abundance 0.00041000000783242285 | Marker 1.0


We obsereve that for each disease only bacteria species and markers which are present at least once are available.
The objective is now to merge these datasets, and come up with the unified comprehensive list of markers and bacteria.

In [34]:
columns = dict()
marker_frames = dict()
abundance_frames = dict()
for disease in ['Cirrhosis', 'Colorectal', 'IBD', 'Obesity', 'T2D', 'WT2D']:
    raw_data = load_data(disease, '.')
    marker = raw_data['marker'].set_index(raw_data['marker'].sampleID)
    marker = marker.drop(columns=['sampleID'])
    marker.columns = [c+'-'+disease for c in marker.columns]
    marker_frames[disease] = marker
    abundance = raw_data['abundance'].set_index(raw_data['abundance'].sampleID)
    abundance = abundance.drop(columns=['sampleID'])
    abundance.columns = marker.columns
    abundance_frames[disease] = abundance
    columns[disease] = marker.columns

In [35]:
marker_joint_df = pd.concat([df for df in marker_frames.values()], axis=1, join="outer")
marker_joint_df = marker_joint_df.fillna(0)
abundance_joint_df = pd.concat([df for df in abundance_frames.values()], axis=1, join="outer")
abundance_joint_df = abundance_joint_df.fillna(0)

for disease in ['Cirrhosis', 'Colorectal', 'IBD', 'Obesity', 'T2D', 'WT2D']:
    marker = marker_joint_df[columns[disease]]
    marker.to_csv('joint_data/marker/marker_{}.csv'.format(disease))
    abundance = abundance_joint_df[columns[disease]]
    abundance.to_csv('joint_data/abundance/abundance_{}.csv'.format(disease))

In [36]:
# check to make sure we did not alter the number of positive and negative samples 
label_dict = {
        # Controls
        'n': 0,
        # Chirrhosis
        'cirrhosis': 1,
        # Colorectal Cancer
        'cancer': 1, 'small_adenoma': 0,
        # IBD
        'ibd_ulcerative_colitis': 1, 'ibd_crohn_disease': 1,
        # T2D and WT2D
        't2d': 1,
        # Obesity
        'leaness': 0, 'obesity': 1,
    }
for disease in ['Cirrhosis', 'Colorectal', 'IBD', 'Obesity', 'T2D', 'WT2D']:
    print(disease)
    raw_data = load_data(disease, '.')
    raw_data['marker'] = raw_data['marker'].set_index(raw_data['marker'].sampleID)
    raw_data['marker'] = raw_data['marker'].drop(columns=['sampleID'])
    labels = np.array([[label_dict[i]] for i in raw_data['marker'].loc['disease'].to_list()])
    labels_after = np.array([[label_dict[i]] for i in marker_joint_df[columns[disease]].loc['disease'].to_list()])
    print('Before: {} / {} | After {} / {}'.format(np.sum(labels), len(labels), np.sum(labels_after), len(labels_after)))

Cirrhosis
Before: 118 / 232 | After 118 / 232
Colorectal
Before: 48 / 121 | After 48 / 121
IBD
Before: 25 / 110 | After 25 / 110
Obesity
Before: 164 / 253 | After 164 / 253
T2D
Before: 170 / 344 | After 170 / 344
WT2D
Before: 53 / 96 | After 53 / 96
